In [12]:
from rich import print
from transitfit import split_lightcurve_file, run_retrieval
from astroquery.mast import Observations as obs
from datetime import datetime
from tabulate import tabulate
from astropy.config import set_temp_cache
from astropy.table import Table
from pandas import DataFrame, read_csv, Categorical
from shutil import rmtree, make_archive, copy
from natsort import natsorted
from math import ceil
import numpy as np
import sys
import os
import random

In [106]:
import pandas as pd
import time

def gen_tess():
    '''Generates all TESS targets which have a fitsfile.'''
    def count_products(tic_id):
        '''Counts the amount of TESS fitsfiles on MAST.'''
        time.sleep(0.01)
        search = obs.query_criteria(dataproduct_type=['timeseries'],
                                    project='TESS',
                                    provenance_name=['SPOC'],
                                    t_exptime=120,
                                    target_name=tic_id).to_pandas()
        search = search[search['t_exptime']==120]
        search = search[~search.dataURL.str.endswith('_dvt.fits')]
        if len(search) is None:
            return 0
        return len(search)
        
    df = read_csv('firefly/data/nasa.csv.gz').dropna(subset=['tic_id']).reset_index(drop=True)
    df['tic_id'] = df['tic_id'].str.replace('TIC ', '')
    #df['Products'] = [count_products(df['tic_id'][row]) for row in range(len(df))]
    df['Products'] = df['tic_id'].apply(count_products)
    df['Transits'] = np.ceil((0.8 * 27.4 / df['pl_orbper']) * df['Products'])
    df['Transits'] = df['Transits'].fillna(-1).astype(int)
    df['pl_name'] = \
            Categorical(df['pl_name'],
            ordered=True,
            categories=natsorted(df['pl_name'].unique()))
    df = df.sort_values('pl_name')
    df.to_csv('nasa.csv.xz', index=False)
    #here = os.path.dirname(os.path.abspath(__file__))
    #df.to_csv(f'{here}/data/Targets/ML_nasa_tess_viable.csv.xz', index=False)

gen_tess()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices